In [1]:
# 프로젝트 디렉토리에서 실행.
# jupyter notebook에서 django shell을 사용할수 있도록 환경변수 설정.

In [1]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()

In [3]:
from polls.models import Question, Choice, SubChoice

# 전체조회
- all()

In [3]:
# Model클래스이름.objects.XXXXXXX
qs = Question.objects.all()
qs

<QuerySet [<Question: 좋아하는 색깔은 무엇입니까?>, <Question: 좋아하는 동물은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요>]>

In [6]:
qs = SubChoice.objects.all()
qs, type(qs)

(<QuerySet [<SubChoice: 검은색이다>, <SubChoice: 파랑색이다>, <SubChoice: 빨강색이다>, <SubChoice: 우리나라다>, <SubChoice: 가고싶은나라다>, <SubChoice: 신분세탁국가다>, <SubChoice: 강아지는 귀엽다>, <SubChoice: 고양이는 귀엽다>, <SubChoice: 거북이는 느리다>, <SubChoice: 보드카의나라다>, <SubChoice: 놀러가고싶은나라다>]>,
 django.db.models.query.QuerySet)

In [5]:
# QuerySet.query : sql문 조회
print(qs.query)

SELECT "polls_subchoice"."sub_id", "polls_subchoice"."sub_choice_id", "polls_subchoice"."description" FROM "polls_subchoice"


In [29]:
for q in qs:
    print(q,type(q))

좋아하는 색깔은 무엇입니까? <class 'polls.models.Question'>
좋아하는 동물은 무엇입니까? <class 'polls.models.Question'>
가고 싶은 나라를 선택하세요 <class 'polls.models.Question'>


In [31]:
# 특정 조회 결과 하나 조회 - indexing
# 범위 조회: slicing / 음수 인덱스는 사용불가.
qs[0], qs[1:]

(<Question: 좋아하는 색깔은 무엇입니까?>,
 [<Question: 좋아하는 동물은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요>])

In [33]:
qs.last(), qs.first()

(<Question: 가고 싶은 나라를 선택하세요>, <Question: 좋아하는 색깔은 무엇입니까?>)

In [34]:
qs = Choice.objects.all()
qs

<QuerySet [<Choice: 검정색>, <Choice: 파랑색>, <Choice: 빨강색>, <Choice: 한국>, <Choice: 미국>, <Choice: 영국>, <Choice: 강아지>, <Choice: 고양이>, <Choice: 거북이>, <Choice: 러시아>, <Choice: 오스트레일리아>]>

In [35]:
qs.order_by('choice_text') # asc

<QuerySet [<Choice: 강아지>, <Choice: 거북이>, <Choice: 검정색>, <Choice: 고양이>, <Choice: 러시아>, <Choice: 미국>, <Choice: 빨강색>, <Choice: 영국>, <Choice: 오스트레일리아>, <Choice: 파랑색>, <Choice: 한국>]>

In [36]:
qs.order_by('-choice_text') # dsc

<QuerySet [<Choice: 한국>, <Choice: 파랑색>, <Choice: 오스트레일리아>, <Choice: 영국>, <Choice: 빨강색>, <Choice: 미국>, <Choice: 러시아>, <Choice: 고양이>, <Choice: 검정색>, <Choice: 거북이>, <Choice: 강아지>]>

In [28]:
qs = Choice.objects.all().order_by('id')
qs

<QuerySet [<Choice: 검정색>, <Choice: 파랑색>, <Choice: 빨강색>, <Choice: 한국>, <Choice: 미국>, <Choice: 영국>, <Choice: 강아지>, <Choice: 고양이>, <Choice: 거북이>, <Choice: 러시아>, <Choice: 스페인>]>

In [21]:
print(qs.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."choice_text" DESC


In [41]:
c = qs.first()
print(type(c))

<class 'polls.models.Choice'>


In [43]:
# 조회한 Model의 속성을 조회 -> 특정 컬럼값 조회
# Choice: id(pk), choice_text, vote, question_id
c.id, c.choice_text, c.vote, c.question, c.question_id

(4, '한국', 0, <Question: 가고 싶은 나라를 선택하세요>, 3)

In [27]:
sc = SubChoice.objects.all().order_by('sub_choice_id')
for c in sc:
    print(c.sub_id, c.sub_choice_id, c.sub_choice, c.description)

1 1 검정색 검은색이다
2 2 파랑색 파랑색이다
3 3 빨강색 빨강색이다
4 4 한국 우리나라다
5 5 미국 가고싶은나라다
6 6 영국 신분세탁국가다
7 7 강아지 강아지는 귀엽다
8 8 고양이 고양이는 귀엽다
9 9 거북이 거북이는 느리다
10 10 러시아 보드카의나라다
11 11 스페인 놀러가고싶은나라다


In [15]:
print(sc.query)

SELECT "polls_subchoice"."sub_id", "polls_subchoice"."sub_choice_id", "polls_subchoice"."description" FROM "polls_subchoice"


In [35]:
for c,s in zip(qs,sc):
    if c.question_id==1:
        print(c.id, c.choice_text, c.vote, c.question, c.question_id, s.sub_id, s.sub_choice, s.description, sep='\t')

1	검정색	0	좋아하는 색깔은 무엇입니까?	1	1	검정색	검은색이다
2	파랑색	0	좋아하는 색깔은 무엇입니까?	1	2	파랑색	파랑색이다
3	빨강색	0	좋아하는 색깔은 무엇입니까?	1	3	빨강색	빨강색이다


# where 절
- filter(조회조건)
    - 조회조건을 만족하는 0개 이상의 행을 조회
- exclude(조회조건)
    - filter의 반대. 조회조건을 만족하지 않는 0개 이상의 행을 조회(not)
- get(조회조건)
    - 조회조건을 만족하는 1개 행을 조회
- filter/exclude : QuerySet으로 반환
- get: Model 객체를 반환

In [6]:
# where id = 1
qs = Question.objects.filter(id=2)
# primary key 컬럼 조회: 컬럼명, pk
qs1 = Question.objects.filter(pk=2)
qs, qs1.first().question_text

(<QuerySet [<Question: 좋아하는 동물은 무엇입니까?>]>, '좋아하는 동물은 무엇입니까?')

In [51]:
print(qs.first().pub_date)

2021-05-25 05:20:01.945546+00:00


In [53]:
quest = Question.objects.get(pk=2) # get(): pk로 조회
print(type(quest), quest)

<class 'polls.models.Question'> 좋아하는 동물은 무엇입니까?


In [7]:
try:
    r = Question.objects.get(pk=1)
    print(r.pub_date)
except:
    print('조회결과가 없습니다')

2021-05-26 02:35:06.990218+00:00


In [8]:
r.question_text

'좋아하는 색깔은 무엇입니까?'

In [9]:
question_id = 2

In [13]:
a = Question.objects.get(pk=question_id)
a.question_text

'좋아하는 동물은 무엇입니까?'

In [58]:
# Question.objects.get(id__gt=1) # id >1

MultipleObjectsReturned: get() returned more than one Question -- it returned 2!

In [59]:
try:
    Question.objects.get(id__gt=1) # id >1
except:
    print('조회결과가 없습니다')

조회결과가 없습니다


In [16]:
# 조회 조건
# qs = Choice.objects.filter(id__gt=7) # id > 7
# qs = Choice.objects.filter(id__gte=7)
qs = Choice.objects.filter(id__lt=3)
# qs = Choice.objects.filter(id__lte=5)
for q in qs:
    print(q.id, q.choice_text)

In [78]:
qs = Question.objects.filter(question_text__startswith = '가고') # q_t like '가고%'
qs = Question.objects.filter(question_text__endswith = '하세요') # q_t like '%하세요'
qs = Question.objects.filter(question_text__contains = '하는') # q_t like '%하는%'
for q in qs:
    print(q.id, q.question_text)

1 좋아하는 색깔은 무엇입니까?
2 좋아하는 동물은 무엇입니까?


In [21]:
qs = Choice.objects.filter(id__in = [2,5,7,9,20]) # where id in [2,5,7,9,20]
qs = Choice.objects.filter(id__range=(0,40)) # between 3 and 7
print(qs.query)
for q in qs:
    print(q.id, q.choice_text)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."id" BETWEEN 0 AND 40
7 강아지
8 고양이
9 거북이
10 러시아
11 스페인
12 한국
13 미국
14 영국
15 검정색
16 파랑색
17 빨강색
18 가
19 나
20 다
21 braveheart
22 remember
23 nonononono
24 shark
25 elf
26 orc
27 kim
28 bae
29 young


### and 조건

In [86]:
qs = Choice.objects.filter(pk=3, choice_text='빨강색')
qs.first().id, qs.first().choice_text

(3, '빨강색')

In [40]:
qs = Choice.objects.filter(question_id=1)
print(qs)
for idx, q in enumerate(qs):
    dic["{}".format(idx)]=q.choice_text
dic.items()

<QuerySet [<Choice: 검정색>, <Choice: 파랑색>, <Choice: 빨강색>]>


dict_items([('2', '빨강색'), ('0', '검정색'), ('1', '파랑색')])

In [32]:
qs = Choice.objects.filter(pk__lt=5, choice_text__contains='색')
qs

<QuerySet []>

In [93]:
# OR조건 Q(), ~Q()
from django.db.models import Q
# qs = Choice.objects.filter(Q(pk__gt=5)|Q(choice_text__contains='색'))
# qs = Choice.objects.filter(Q(pk__gt=5)|~Q(choice_text__contains='색')) # Not
qs = Choice.objects.filter(~Q(pk=5))
print(qs.query)
for q in qs:
    print(q.id, q.choice_text)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" = 5)
1 검정색
2 파랑색
3 빨강색
4 한국
6 영국
7 강아지
8 고양이
9 거북이
10 러시아
11 스페인


In [95]:
qs = Choice.objects.exclude(pk=5)
qs = Choice.objects.exclude(pk__lt=5)
print(qs.query)
for q in qs:
    print(q.id, q.choice_text)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" < 5)
5 미국
6 영국
7 강아지
8 고양이
9 거북이
10 러시아
11 스페인


# 컬럼 선택
select 컬럼, 컬럼
- values(field이름,....)
    - 개별 조회결과를 Dictionary에 넣어서 반환.

In [103]:
qs = Choice.objects.all().values('id', 'choice_text').order_by('-id')
print(qs.query)
for q in qs:
    print(q['id'], q['choice_text'])

SELECT "polls_choice"."id", "polls_choice"."choice_text" FROM "polls_choice" ORDER BY "polls_choice"."id" DESC
11 스페인
10 러시아
9 거북이
8 고양이
7 강아지
6 영국
5 미국
4 한국
3 빨강색
2 파랑색
1 검정색


# 집계
- 단순 집계
    - aggregate(집계함수(),...)
- Group by집계
    - values(그룹으로 묶을 컬럼).annotate(집계함수(),...)
- 결과를 dictionary로 반환


In [6]:
# 조회결과 행수 조회
qs = Choice.objects.all()
qs = Choice.objects.filter(id__lt=3)
len(qs), qs.count()

(2, 2)

In [7]:
from django.db.models import Count, Sum, Avg, Min, Max, StdDev, Variance

In [12]:
# dic = Choice.objects.aggregate(Count('id')) # sql: select count('id') form choice
dic = Choice.objects.aggregate(Count('id'), Count('choice_text'), Sum('vote'), Avg('vote'), Min('vote'), Max('vote'), StdDev('vote'),Variance('vote'))
type(dic), dic

(dict,
 {'id__count': 11,
  'choice_text__count': 11,
  'vote__sum': 331,
  'vote__avg': 30.09090909090909,
  'vote__min': 2,
  'vote__max': 82,
  'vote__stddev': 27.337801153363202,
  'vote__variance': 747.3553719008264})

In [14]:
dic = Question.objects.aggregate(Min('pub_date'), Max('pub_date'))
dic

{'pub_date__min': datetime.datetime(2021, 5, 25, 5, 18, 55, 578487, tzinfo=<UTC>),
 'pub_date__max': datetime.datetime(2021, 5, 25, 5, 20, 55, 995019, tzinfo=<UTC>)}

In [16]:
dic['pub_date__min'],dic['pub_date__max'] 

(datetime.datetime(2021, 5, 25, 5, 18, 55, 578487, tzinfo=<UTC>),
 datetime.datetime(2021, 5, 25, 5, 20, 55, 995019, tzinfo=<UTC>))

In [17]:
print(dic['pub_date__min'],dic['pub_date__max'])

2021-05-25 05:18:55.578487+00:00 2021-05-25 05:20:55.995019+00:00


In [19]:
dic['pub_date__min'].strftime('%Y년 %M월 %d일')

'2021년 18월 25일'

In [28]:
# where + 집계
dic = Choice.objects.filter(id__gt = 5).aggregate(Sum('vote'))
dic

{'vote__sum': 53}

In [34]:
# group by 
# choice => 질문별 vote의 합계, 평균
# 그룹별 집계: dictionary, 최종반환: QuerySet
qs = Choice.objects.values('question_id').annotate(Sum('vote'), Avg('vote'))
qs

<QuerySet [{'question_id': 1, 'vote__sum': 141, 'vote__avg': 47.0}, {'question_id': 2, 'vote__sum': 36, 'vote__avg': 12.0}, {'question_id': 3, 'vote__sum': 154, 'vote__avg': 30.8}]>

In [35]:
for d in qs:
    print(d)

{'question_id': 1, 'vote__sum': 141, 'vote__avg': 47.0}
{'question_id': 2, 'vote__sum': 36, 'vote__avg': 12.0}
{'question_id': 3, 'vote__sum': 154, 'vote__avg': 30.8}


In [37]:
for d in qs:
    print(f"질문:{d['question_id']}, 투표합계:{d['vote__sum']}, 투표평균:{d['vote__avg']}")

질문:1, 투표합계:141, 투표평균:47.0
질문:2, 투표합계:36, 투표평균:12.0
질문:3, 투표합계:154, 투표평균:30.8


# 테이블간의 관계

## 자식테이블에서 부모 테이블의 값을 조회

In [46]:
c1 = Choice.objects.get(pk=3)
c1.question, c1.question_id, c1.choice_text, type(c1)

(<Question: 좋아하는 색깔은 무엇입니까?>, 1, '빨강색', polls.models.Choice)

In [48]:
# 모델에서 속성들 조회 => 각 컬럼들 값 조회.
c1.id, c1.pk, c1.choice_text, c1.vote, c1.question, c1.question_id

(3, 3, '빨강색', 56, <Question: 좋아하는 색깔은 무엇입니까?>, 1)

In [51]:
sc1 = SubChoice.objects.get(pk=3)
sc1

<SubChoice: 빨강색이다>

In [52]:
sc1.sub_id, sc1.sub_choice, sc1.sub_choice_id, sc1.description

(3, <Choice: 빨강색>, 3, '빨강색이다')

## 부모테이블에서 자식테이블 값을 조회

In [54]:
q = Question.objects.get(pk=3)
q, type(q)

(<Question: 가고 싶은 나라를 선택하세요>, polls.models.Question)

In [55]:
q.id, q.question_text, q.pub_date

(3,
 '가고 싶은 나라를 선택하세요',
 datetime.datetime(2021, 5, 25, 5, 20, 55, 995019, tzinfo=<UTC>))

In [60]:
c = q.choice_set
type(c)
c.all()

<QuerySet [<Choice: 한국>, <Choice: 미국>, <Choice: 영국>, <Choice: 러시아>, <Choice: 스페인>]>

In [61]:
c.filter(choice_text = '한국')

<QuerySet [<Choice: 한국>]>

# insert / update / delete
- insert/update
    - 모델객체(insert할 내용을가진).save()
- delete
    - 모델객체(삭제할 pk를 가진).delete()

In [66]:
# insert
qs = Question.objects.all()
for q in qs:
    print(q.pk)

1
2
3
4


In [63]:
from datetime import datetime, timedelta

In [64]:
# pk: id=자동증가로 설정되있으므로 생략
new_q = Question()

new_q.question_text='좋아하는 가수는?'
new_q.pub_date = datetime.now()
new_q.question_text, new_q.pub_date, new_q.pk

('좋아하는 가수는?', datetime.datetime(2021, 5, 26, 11, 22, 16, 157508), None)

In [80]:
# pk: id=자동증가로 설정되있으므로 생략
new_q = Question()
new_q.id = 1
new_q.question_text='삭제후 삽입'
new_q.pub_date = datetime.now()
new_q.question_text, new_q.pub_date, new_q.pk

('삭제후 삽입', datetime.datetime(2021, 5, 26, 11, 35, 5, 518939), 1)

In [81]:
new_q.save()

C:\Users\mein0\anaconda3\envs\django\lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField Question.pub_date received a naive datetime (2021-05-26 11:35:05.518939) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [82]:
qs = Question.objects.all()
for q in qs:
    print(q.pk, q.question_text, q.pub_date)

1 삭제후 삽입 2021-05-26 02:35:06.990218+00:00
2 좋아하는 동물은 무엇입니까? 2021-05-25 05:20:01.945546+00:00
3 가고 싶은 나라를 선택하세요 2021-05-25 05:20:55.995019+00:00
4 좋아하는가수는 누구입니까? 2021-02-15 02:30:11.436359+00:00


In [75]:
q = Question.objects.get(pk=4)
q.question_text = "좋아하는가수는 누구입니까?"
q.pub_date = datetime.now() - timedelta(days=100)
print(q.question_text, q.pub_date)
q.save()

좋아하는가수는 누구입니까? 2021-02-15 11:30:11.436359


C:\Users\mein0\anaconda3\envs\django\lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField Question.pub_date received a naive datetime (2021-02-15 11:30:11.436359) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [77]:
# 삭제
d_q = Question()
d_q.id = 1
d_q.id, d_q.question_text, d_q.pub_date

(1, '', None)

In [78]:
d_q.delete()

(7, {'polls.SubChoice': 3, 'polls.Choice': 3, 'polls.Question': 1})

In [83]:
Choice.objects.all()

<QuerySet [<Choice: 한국>, <Choice: 미국>, <Choice: 영국>, <Choice: 강아지>, <Choice: 고양이>, <Choice: 거북이>, <Choice: 러시아>, <Choice: 스페인>]>

In [86]:
# 특정 조건의 데이터들을 삭제 => 조회
# choice_text에 "국"이 들어간 것들 삭제
qs = Choice.objects.filter(choice_text__endswith='국')
for q in qs:
    q.delete()

In [87]:
Choice.objects.all()

<QuerySet [<Choice: 강아지>, <Choice: 고양이>, <Choice: 거북이>, <Choice: 러시아>, <Choice: 스페인>]>

# 직접 SQL문을 실행
- 모델.objects.raw('select문')
    - RawQuerySet

In [91]:
rq = Choice.objects.raw('select * from polls_choice')
rq , rq.query

(<RawQuerySet: select * from polls_choice>,
 <RawQuery: select * from polls_choice>)

In [93]:
for q in rq:
    print(q,type(q))

강아지 <class 'polls.models.Choice'>
고양이 <class 'polls.models.Choice'>
거북이 <class 'polls.models.Choice'>
러시아 <class 'polls.models.Choice'>
스페인 <class 'polls.models.Choice'>
